In [1]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
import json
matplotlib.rcParams["image.interpolation"] = 'nearest'
import matplotlib.pyplot as plt
import os
from PIL import Image
import tensorflow as tf
from glob import glob
from tqdm import tqdm
from tifffile import imread
import struct
import pandas as pd
import cv2
from csbdeep.utils import Path, normalize

from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist.models import Config2D, StarDist2D, StarDistData2D

In [2]:
def load_model(model_path: str) -> StarDist2D:
    # Load StarDist model weights, configurations, and thresholds
    with open(model_path + '\\config.json', 'r') as f:
        config = json.load(f)
    with open(model_path + '\\thresholds.json', 'r') as f:
        thresh = json.load(f)
    model = StarDist2D(config=Config2D(**config), basedir=model_path, name='offshoot_model')
    model.thresholds = thresh
    print('Overriding defaults:', model.thresholds, '\n')
    model.load_weights(model_path + '\\weights_best.h5')
    return model

In [ ]:
model = load_model(r'\\10.99.68.178\andreex\data\Stardist\qupath_training_annotations\models\monkey_add_fallopian_test_12_7_23_lr_0.001_epochs_200_pt_40')

In [ ]:
pthHE = r'\\10.99.68.178\andreex\data\Stardist\Testing\monkey\tiles'
pthmasks = r'\\10.99.68.178\andreex\data\Stardist\Testing\monkey\qupath\ground_truth\masks'

HE_tiles_pths = sorted(glob(os.path.join(pthHE,'*.tif')))
mask_tiles_pths = sorted(glob(os.path.join(pthmasks,'*.tif')))

# not done with all tiles yet
#pthHE_new = r'\\10.99.68.178\andreex\data\Stardist\Testing\monkey\12_5_new_monkey_tiles'
#pthHE_new_pths = sorted(glob(os.path.join(pthHE_new,'*.tif')))
# = [os.path.basename(pth) for pth in pthHE_new_pths]
#HE_tiles_pths = [name for name in HE_tiles_pths if name[58:] not in pthHE_new_names]

print(len(HE_tiles_pths))

In [ ]:
HE_tiles = list(map(imread,HE_tiles_pths))
mask_tiles = list(map(imread,mask_tiles_pths))

In [ ]:
print(len(HE_tiles))
print(len(mask_tiles))

In [ ]:
HE_tiles_default = [normalize(x) for x in HE_tiles]  # just divide by 255 to get into (0,1) range
mask_tiles_default = [fill_label_holes(y) for y in mask_tiles]  # fills holes in annotations if there are any

HE_tiles = [x/255 for x in HE_tiles]  # just divide by 255 to get into (0,1) range
mask_tiles = [fill_label_holes(y) for y in mask_tiles]  # fills holes in annotations if there are any

In [ ]:
mask_val_pred = [model.predict_instances(x, n_tiles=model._guess_n_tiles(x), show_tile_progress=False)[0]
              for x in tqdm(HE_tiles)]

In [ ]:
#PLot image and label for some images - sanity check
def show_tile_segmented(tile, segmented, **kwargs):
    """Plot large image at different resolutions."""
    fig, ax = plt.subplots(1, 2, figsize=(16/2, 8/2))

    # Plot the original image on the left
    ax[0].imshow(tile, **kwargs)

    # Plot the cropped image on the right
    ax[1].imshow(segmented, **kwargs)

    ax[0].axis('off')
    ax[1].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
#Random color map labels
np.random.seed(42)
lbl_cmap = random_label_cmap()

In [ ]:
ind = 1

cropped_img = HE_tiles[ind]
cropped_lbl_gt = mask_tiles[ind]
cropped_lbl_pred = mask_val_pred[ind]


fig, ax = plt.subplots(1,3,figsize = (24, 8))
ax[0].imshow(cropped_img)
ax[0].axis('off')
ax[0].set_title('H&E')

ax[1].imshow(cropped_img)
ax[1].imshow(cropped_lbl_gt, cmap=lbl_cmap, alpha=0.5)
ax[1].axis('off')
ax[1].set_title('Ground Truth')

ax[2].imshow(cropped_img)
ax[2].imshow(cropped_lbl_pred, cmap=lbl_cmap, alpha=0.5)
ax[2].axis('off')
ax[2].set_title('Predicted')

In [ ]:
import sys
sys.path.insert(1, r'C:\Users\Donald\PycharmProjects\monkey_nuc_segment\stardist\monkey')
from scoring_for_donald import TileSetScorer, ScoringSubroutine

nms = [os.path.basename(file) for file in HE_tiles_pths]

# taus = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
taus = [0.5]

scores = TileSetScorer(nms, mask_tiles, mask_val_pred, taus)

In [ ]:
scores.score_set()

In [ ]:
results = scores.score_set()
names = results['Image']

names = [name.split(".")[0][21:] for name in names]

for i in range(len(names)):
    #if len(names[i]) > 6:
    names[i] = names[i][:2]

f1_scores = results['F1 Score']

In [ ]:
bar_width = 0.25  # Adjust the width of each bar as needed
index = np.arange(len(names))

# Plotting the bars
fig = plt.figure(figsize=(25, 10))
fig.set_facecolor('white')

plt.bar(index, f1_scores, color='blue', label='Newest Model')

#plt.bar(index, f1_scores, color='blue', width=bar_width, label='Newest Model')
#plt.bar(index + bar_width, f1_scores_justin, color='orange', width=bar_width, label="Previous model")
#plt.bar(index + 2 * bar_width, f1_scores_default, color='green', width=bar_width, label='Default H&E')

plt.xlabel("Tile Name", fontsize=20)
plt.ylabel("F1 Score", fontsize=20)
# plt.title("F1 Scores in Testing Tiles (tau = 0.6)", fontsize=28)
plt.axhline(y=0.7, linestyle='--', color='red', label='Threshold = 0.7')
plt.ylim(0, 1)
plt.xticks(index + bar_width, names)  # Set x-axis labels to tile names
plt.legend(fontsize=20)
plt.show()

In [ ]:
results[['Image', 'F1 Score']]

In [ ]:
raw_vals = results[['Image', 'TP', 'FP', 'FN']]
raw_vals

In [ ]:
def find_first_unique_indices(lst):
    unique_indices = {}
    result = []

    for i, value in enumerate(lst):
        if value not in unique_indices:
            unique_indices[value] = i

    for value in lst:
        result.append(unique_indices[value])

    return result

result_list = find_first_unique_indices(names)
print(result_list)

In [ ]:
print(len(np.unique(result_list)))
print(len(names))

In [ ]:
print([names[i] for i in np.unique(result_list)])

In [ ]:
len(raw_vals)

In [ ]:
seen = []
names = []

TPs = [0 for _ in range(len(raw_vals))]
FPs = [0 for _ in range(len(raw_vals))]
FNs = [0 for _ in range(len(raw_vals))]

for i in range(len(raw_vals)):
    row = raw_vals.iloc[i]
    print(num)
    num = result_list[i]
    if num not in seen:
        names.append(raw_vals['Image'][i])
        seen.append(num)

    TPs[num] += row['TP']
    FPs[num] += row['FP']
    FNs[num] += row['FN']

In [ ]:
TPs

In [ ]:
raw_vals

In [ ]:
print(names)
print(TPs)

In [ ]:
print(len(names))

In [ ]:
organ_names = []
for i in range(len(names)):
    nm = names[i][21:-4]
    if nm[-1] == '1':
        nm = nm[:-1]
    organ_names.append(nm)
    print(nm)

In [ ]:
print(len(organ_names))

In [ ]:
organ_names

In [ ]:
TPs = [val for val in TPs if val != 0]
FPs = [val for val in FPs if val != 0]
FNs = [val for val in FNs if val != 0]

In [ ]:
len(TPs)

In [ ]:
new_df = pd.DataFrame({'Organ': organ_names, 'TP': TPs, 'FP': FPs, 'FN': FNs})
new_df

In [ ]:
def get_f1(tp, fp, fn):
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    avg_precision = tp / (tp + fp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, avg_precision, f1

In [ ]:
precisions = []
recalls = []
f1s = []

for i in range(len(new_df)):
    row = new_df.iloc[i]

    tp = row['TP']
    fp = row['FP']
    fn = row['FN']

    precision, recall, avg_precision, f1 = get_f1(tp, fp, fn)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

In [ ]:
row = new_df.iloc[11]

tp = row['TP']
fp = row['FP']
fn = row['FN']
print([tp, fp, fn])

In [ ]:
new_df['Recall'] = recalls
new_df['Precision'] = precisions
new_df['F1'] = f1s

new_df

In [ ]:
bar_width = 0.25  # Adjust the width of each bar as needed
index = np.arange(len(new_df['Organ']))

# Plotting the bars
fig = plt.figure(figsize=(25, 10))
fig.set_facecolor('white')

plt.bar(new_df['Organ'], new_df['F1'], color='blue', label='Newest Model')

#plt.bar(index, f1_scores, color='blue', width=bar_width, label='Newest Model')
#plt.bar(index + bar_width, f1_scores_justin, color='orange', width=bar_width, label="Previous model")
#plt.bar(index + 2 * bar_width, f1_scores_default, color='green', width=bar_width, label='Default H&E')

plt.xlabel("Organ", fontsize=20)
plt.ylabel("F1 Score", fontsize=20)
# plt.title("F1 Scores in Testing Tiles (tau = 0.6)", fontsize=28)
plt.axhline(y=0.7, linestyle='--', color='red', label='Threshold = 0.7')
plt.ylim(0, 1)
plt.xticks(index + bar_width, new_df['Organ'])  # Set x-axis labels to tile names
plt.legend(fontsize=20)
plt.show()

In [ ]:
bar_width = 0.25  # Adjust the width of each bar as needed
index = np.arange(len(names))

# Plotting the bars
fig = plt.figure(figsize=(25, 10))
fig.set_facecolor('white')

plt.bar(index, f1_scores, color='blue', label='Newest Model')


plt.xlabel("Tile Name", fontsize=20)
plt.ylabel("F1 Score", fontsize=20)
# plt.title("F1 Scores in Testing Tiles (tau = 0.6)", fontsize=28)
plt.axhline(y=0.7, linestyle='--', color='red', label='Threshold = 0.7')
plt.ylim(0, 1)
plt.xticks(index + bar_width, names)  # Set x-axis labels to tile names
plt.legend(fontsize=20)
plt.show()

In [ ]:
from scoring_for_donald import TileSetScorer, ScoringSubroutine

In [ ]:
def calc_iou_for_each_nucleus(masks1, masks2) -> list[float]:
    iou_scores = []

    for mask1, mask2 in zip(masks1, masks2):
        intersection = np.logical_and(mask1, mask2)
        union = np.logical_or(mask1, mask2)
        intersection_area = np.sum(intersection)
        union_area = np.sum(union)

        if union_area > 0:
            iou = intersection_area / union_area
            iou_scores.append(iou)
        else:
            iou_scores.append(0.0)

    return iou_scores

In [ ]:
test_gt = mask_tiles[0]
test_pred = mask_val_pred[0]

test_iou = calc_iou_for_each_nucleus(test_gt, test_pred)
print(test_iou)

In [ ]:
def plot_masks(masks1: list[np.ndarray], masks2: list[np.ndarray]):
    num_masks1 = len(masks1)
    num_masks2 = len(masks2)

    fig, axs = plt.subplots(num_masks1, num_masks2, figsize=(12, 8))

    for i in range(num_masks1):
        for j in range(num_masks2):
            axs[i, j].imshow(masks1[i], cmap='Blues', alpha=0.5)
            axs[i, j].imshow(masks2[j], cmap='Reds', alpha=0.5)
            axs[i, j].set_xticks([])
            axs[i, j].set_yticks([])

    # Add legends
    axs[0, 0].legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

    plt.tight_layout()
    plt.show()

plot_masks(test_gt, test_pred)

In [ ]:
for i, mask1 in enumerate(masks_nuclei1):
    for j, mask2 in enumerate(masks_nuclei2):
        iou_matrix[i, j] = calc_iou_for_each_nucleus([mask1], [mask2])[0]

plot_iou_heatmap(iou_matrix, labels1=['Nucleus 1', 'Nucleus 2', ...], labels2=['Nucleus 1', 'Nucleus 2', ...])